# How similar is the text?

In [1]:
import os, sys, spacy

import pandas as pd
# import gensim.downloader as api

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from feature_extraction import SpacyFeatureExtraction
# from data_processing import DataProcessing

In [2]:
# load_nlp_model = spacy.load("en_core_web_md")
# pretrained_word_two_vec_model = api.load('word2vec-google-news-300')

In [3]:
mappings = {
    "I predict it will be sunny on 4 April 2025." : ["It was sunny on 4 April 2025.", "It was not sunny on 4 April 2025."],
    "Apple's stock will rise by 10%." : ["Apple's stock rose by 11%.", "Apple's stock rose by 900%.", "Apple's stock rose by 10%."]
}

mappings = {
    "Tesla's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 10%.", "Apple's stock rose by 900%."]
}

mappings = {
    "Tesla's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 10%.", "Apple's stock rose by 900%."],
    "Apple's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 900%.", "Apple's stock rose by 10%."]
}

In [4]:
load_nlp_model = spacy.load("en_core_web_sm")

all_mappings = {} # Mapping of predictions to observations
pom_mappings = {} # Mapping of predictions to observations to metrics/scoresl pos can be misleading

oms = []
for prediction in mappings.keys():
    print(f"Prediction: {prediction}")

    om_mappings = {} # Mapping of observations to metrics/scores

    for observation in mappings[prediction]:
        print(f"    Observation: {observation}")

        # Calculate the similarity score
        prediction_doc = load_nlp_model(prediction)
        observation_doc = load_nlp_model(observation)
        similarity = prediction_doc.similarity(observation_doc)
        print(f"    Similarity: {similarity}")
        om_mappings[observation] = [similarity]
        print()   
        # print(f"    OM Mapping: {om_mappings}")
    pom_mappings[prediction] = om_mappings


Prediction: Tesla's stock will rise by 10%.
    Observation: Apple's stock will rise by 10%.
    Similarity: 0.9796735644340515

    Observation: Apple's stock rose by 11%.
    Similarity: 0.8603723049163818

    Observation: Apple's stock rose by 10%.
    Similarity: 0.8691261410713196

    Observation: Apple's stock rose by 900%.
    Similarity: 0.8487552404403687

Prediction: Apple's stock will rise by 10%.
    Observation: Apple's stock will rise by 10%.
    Similarity: 1.0

    Observation: Apple's stock rose by 11%.
    Similarity: 0.8647665977478027

    Observation: Apple's stock rose by 900%.
    Similarity: 0.8541026711463928

    Observation: Apple's stock rose by 10%.
    Similarity: 0.8756679892539978



/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_11808/312081125.py:18: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


In [5]:
# pom_mappings

In [6]:
# Flatten the dictionary into a list of rows
data = []
for prediction, observations in pom_mappings.items():
    for observation, scores in observations.items():
        data.append([prediction, observation, scores[0]])

# Create the DataFrame
df = pd.DataFrame(data, columns=['Prediction', 'Observation', 'Spacy Similarity'])

# Display the DataFrame
df

,Prediction,Observation,Spacy Similarity
0,Tesla's stock will rise by 10%.,Apple's stock will rise by 10%.,0.979674
1,Tesla's stock will rise by 10%.,Apple's stock rose by 11%.,0.860372
2,Tesla's stock will rise by 10%.,Apple's stock rose by 10%.,0.869126
3,Tesla's stock will rise by 10%.,Apple's stock rose by 900%.,0.848755
4,Apple's stock will rise by 10%.,Apple's stock will rise by 10%.,1.000000
5,Apple's stock will rise by 10%.,Apple's stock rose by 11%.,0.864767
6,Apple's stock will rise by 10%.,Apple's stock rose by 900%.,0.854103
7,Apple's stock will rise by 10%.,Apple's stock rose by 10%.,0.875668


Comparison
1. No focus on #s, more on words

In [7]:
prediction_doc = load_nlp_model("10%")
observation_doc = load_nlp_model("10.5%")
observation_doc_2 = load_nlp_model("11%")
similarity = prediction_doc.similarity(observation_doc)
similarity

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_11808/1801851099.py:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


0.925104558467865

In [8]:
similarity = prediction_doc.similarity(observation_doc_2)
similarity

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_11808/4256136358.py:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc_2)


0.9595414400100708

In [9]:
prediction_doc = load_nlp_model("Apple's stock rose by 10%.")
observation_doc = load_nlp_model("Apple's stock rose by 11%.")
similarity = prediction_doc.similarity(observation_doc)
similarity

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_11808/3994610451.py:3: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


0.9899830222129822

In [10]:
pdv = prediction_doc.vector # 10
pdv

array([-9.69051644e-02,  4.43708450e-02, -3.03836912e-02, -2.53964216e-04,
        3.76424551e-01, -8.68973136e-03,  3.35591853e-01,  2.44690582e-01,
        5.41071951e-01, -3.75136286e-01, -5.61027229e-02,  4.03797626e-01,
       -3.96669418e-01,  6.21899724e-01, -1.49399310e-01,  4.20594901e-01,
       -6.60013497e-01,  4.23253179e-02,  1.75270289e-01,  2.39881963e-01,
       -3.27159643e-01,  2.70389378e-01,  1.91664860e-01, -1.41071409e-01,
        4.56980646e-01, -2.14353502e-02,  1.68063372e-01, -1.47116780e-01,
        2.89462246e-02,  2.59644300e-01,  1.40097201e-01, -3.19258541e-01,
        1.61395758e-01, -5.15149176e-01, -2.74461329e-01,  1.34547830e-01,
       -1.52158633e-01, -2.81525981e-02, -8.63275900e-02,  2.68475771e-01,
        2.45269939e-01, -6.46582991e-03,  2.96593547e-01,  4.67894405e-01,
        7.09100217e-02, -3.24479729e-01, -6.74480051e-02,  6.88518107e-01,
        1.01036370e-01, -3.16862643e-01,  7.60790408e-02,  7.62370974e-02,
        3.15953270e-02, -

In [11]:
odv = observation_doc.vector # 10.5
odv

array([-0.1375021 ,  0.05036664, -0.08244718, -0.09218702,  0.4259199 ,
       -0.02057135,  0.3527473 ,  0.24258173,  0.58889395, -0.4079244 ,
       -0.0078516 ,  0.41594058, -0.30752796,  0.65036994, -0.10828082,
        0.39248484, -0.7268542 ,  0.08076883,  0.2074768 ,  0.31021369,
       -0.31106988,  0.31786174,  0.10419352, -0.14996365,  0.45799989,
        0.00623123,  0.1005058 , -0.1621193 ,  0.03166847,  0.29230043,
        0.16873193, -0.37811226,  0.21399504, -0.57316744, -0.24211708,
        0.13694425, -0.14415772, -0.05136675, -0.07773649,  0.25697094,
        0.31111857, -0.05671643,  0.27724865,  0.45298132,  0.06111433,
       -0.29012573, -0.06525774,  0.6740903 ,  0.10756557, -0.33304495,
       -0.01102847,  0.11045712, -0.02698904, -0.68040615,  0.29783314,
       -0.12875497,  0.17033224, -0.34667563,  0.5682665 , -0.13759264,
       -0.50634605, -0.75957036,  0.00584182, -0.26786065, -0.09881514,
       -0.14349844,  0.46282738, -0.4658656 , -0.2560179 ,  0.21

[Five most popular similarity measures implementation in python](https://dataaspirant.com/five-most-popular-similarity-measures-implementation-in-python/)

In [12]:
from math import*

def square_rooted(x):

    return round(sqrt(sum([a*a for a in x])),3)

def cosine_similarity(x,y):

    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return round(numerator/float(denominator),3)

cosine_similarity(pdv, odv)

np.float32(0.99)

In [13]:
prediction_doc = load_nlp_model("ten")
observation_doc = load_nlp_model("ten point five")
observation_doc_2 = load_nlp_model("eleven")

similarity = prediction_doc.similarity(observation_doc)
similarity

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_11808/3520882758.py:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


0.6815696358680725

In [14]:
similarity = prediction_doc.similarity(observation_doc_2)
similarity

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_11808/4256136358.py:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc_2)


0.6970446705818176

In [17]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example sentences
sentence1 = "Apple's stock rose by 10%."
sentence2 = "Apple's stock rose by 10.5%."

# Tokenize the sentences
tokens1 = tokenizer.tokenize(sentence1)
tokens2 = tokenizer.tokenize(sentence2)

# Add [CLS] and [SEP] tokens
tokens = ['[CLS]'] + tokens1 + ['[SEP]'] + tokens2 + ['[SEP]']
print("Token:", tokens)

# Convert tokens to input IDs
input_ids = tokenizer.convert_tokens_to_ids(tokens)

# Display the tokens and input IDs
print("Input IDs:", input_ids)

from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Convert tokens to input IDs
input_ids1 = torch.tensor(tokenizer.convert_tokens_to_ids(tokens1)).unsqueeze(0)  # Batch size 1
input_ids2 = torch.tensor(tokenizer.convert_tokens_to_ids(tokens2)).unsqueeze(0)  # Batch size 1

# Obtain the BERT embeddings
with torch.no_grad():
    outputs1 = model(input_ids1)
    outputs2 = model(input_ids2)
    embeddings1 = outputs1.last_hidden_state[:, 0, :]  # [CLS] token
    embeddings2 = outputs2.last_hidden_state[:, 0, :]  # [CLS] token

# Calculate similarity
similarity_score = cosine_similarity(embeddings1, embeddings2)
print("Similarity Score:", similarity_score)

Token: ['[CLS]', 'apple', "'", 's', 'stock', 'rose', 'by', '10', '%', '.', '[SEP]', 'apple', "'", 's', 'stock', 'rose', 'by', '10', '.', '5', '%', '.', '[SEP]']
Input IDs: [101, 6207, 1005, 1055, 4518, 3123, 2011, 2184, 1003, 1012, 102, 6207, 1005, 1055, 4518, 3123, 2011, 2184, 1012, 1019, 1003, 1012, 102]
Similarity Score: [[0.947673]]
